# Part 1: 4 x 4 Prime Magic Square


In [2]:
# first lets get all the primes up to 2026 using standard seive
MAX_NUM = 2026
primes = set()
unseen = [1] * (MAX_NUM + 1)
for start in range(2, MAX_NUM + 1):
    cur_idx = start
    if unseen[cur_idx]:
        primes.add(start)
        while cur_idx <= MAX_NUM:
            unseen[cur_idx] = 0
            cur_idx += start
primes = list(primes)
print(len(primes))

306


So we have like $\boxed{|P| = 306}$ primes to work with.

$N$ distinct columns of $N$ numbers $\implies$ If the squares were super well behaved we
could have 17 rows of 17 primes. This is unlikely.

Note since all primes except 2 are odd, and 2026 is even, we can only hope
to get a prime magic square for *even* $N$. The only candidates for the size of a magic primes square for 2026 are the evens from 2 to 16.

We could use an IP solver to try and make these... It turns out even with a suboptimal model, we can find the dimensions that work fairly quickly.

Let the primes be denoted $P = \{p_1, p_2, \ldots p_K\}$. Suppose we have an $N \times N$ magic square. Let $X_{ij}$ be the integer variable that takes on the value the value of the prime
in the $i$-th row and $j$-th column. We need to enforce that the row and column sums are all each 2026.

$$\sum_{i}X_{ij} = 2026 \quad \forall j \in [N] \qquad (\text{column sums 2026})$$
$$\sum_{j}X_{ij} = 2026 \quad \forall i \in [N]\qquad (\text{row sums 2026})$$
$$\sum_{i}X_{ii} = 2026\qquad (\text{main diag sums 2026})$$
$$\sum_{i}X_{i,N + 1- i} = 2026\qquad (\text{anti diag sums 2026})$$

Lastly, we need to enforce that each $X_{ij}$ is a prime from our set and that there are no repeated primes. To do this, we introduce the auxiliary binary variable $Y_{ijk}$ which is 1 when the $(i,j)$-entry of the square is the $k$-th prime. 


$$\sum_{k\in [K]}Y_{ijk} = 1 \quad \forall (i,j) \in [N] \times [N] \qquad (\text{each entry must be a single prime})$$
$$X_{ij} = \sum_{k\in [K]}p_k\cdot Y_{ijk} \quad \forall (i,j) \in [N] \times [N] \qquad (\text{Define $X_{ij}$ as the sum of indicator variables})$$
$$\sum_{i, j} Y_{ijk} = 1 \quad \forall k \in [K]\qquad (\text{All primes used are unique})$$

In [3]:
import pyomo.environ as pyo
import numpy as np


def prime_magic_square(N, magic_num):
    # attempts to find a prime magic square of size N x N

    # we need to define the index sets which will be the locations of the numbers in the square
    model = pyo.ConcreteModel()

    #  ==============  INDEX SETS =================================================
    model.N = pyo.RangeSet(N)
    # print(model.N.value)
    model.P = pyo.RangeSet(0, (len(primes) - 1))

    #  ==============  Variables =====================================================
    model.x = pyo.Var(model.N, model.N, domain=pyo.PositiveIntegers)
    model.y = pyo.Var(model.N, model.N, model.P, domain=pyo.Binary)

    # ============= Constraints =================================================
    model.cons = pyo.ConstraintList()

    # every value must be from the primes
    for i in model.N:
        for j in model.N:
            LHS = sum(model.y[i, j, k] * primes[k] for k in model.P)
            model.cons.add(LHS == model.x[i, j])
            model.cons.add(sum(model.y[i, j, k] for k in model.P) == 1)

    # row and col sums are magic
    for r in model.N:
        row_sum = sum(model.x[r, c] for c in model.N)
        model.cons.add(row_sum == magic_num)
    for c in model.N:
        col_sum = sum(model.x[r, c] for r in model.N)
        model.cons.add(col_sum == magic_num)

    # all cells unique
    for k in model.P:
        model.cons.add(sum(model.y[r, c, k] for r in model.N for c in model.N) <= 1)

    # both diags are magic
    diag_1 = sum(model.x[r, r] for r in model.N)
    diag_2 = sum(model.x[r, N - r + 1] for r in model.N)
    model.cons.add(diag_1 == magic_num)
    model.cons.add(diag_2 == magic_num)

    # ============== Objective ===========================================
    # want existence, no objective function
    model.OBJ = pyo.Objective(expr=0)

    # ============= Solve and get results =========================
    solver = pyo.SolverFactory("cplex")  # or cplex, glpk, ipopt, etc.
    results = solver.solve(model)

    tc = results.solver.termination_condition
    ss = results.solver.status

    if tc in (
        pyo.TerminationCondition.optimal,
        pyo.TerminationCondition.feasible,
    ):
        print(f"Feasible solution found for {N = } \n")

    elif tc == pyo.TerminationCondition.infeasible:
        print(f"Infeasible. No solution for {N = } \n")
        return

    else:
        print(f"Solver ended with condition: {tc}, status: {ss}")
        return

    # extract solution matrix
    solu = [[0] * N for _ in range(N)]
    for r in model.N:
        for c in model.N:
            solu[r - 1][c - 1] = int(model.x[r, c].value)

    # print solution and 2x result is magix
    show_square_matrix(solu)
    return


# Helper function to print out the solution (AI was used to produce this. It is strictly for visualization)
def show_square_matrix(A):
    A = np.asarray(A, dtype=int)
    n, m = A.shape
    assert n == m, "Matrix must be square"

    row_sums = A.sum(axis=1)
    col_sums = A.sum(axis=0)
    diag_main = np.trace(A)
    diag_anti = np.trace(np.fliplr(A))

    # compute column width
    max_val = max(
        np.max(np.abs(A)),
        np.max(np.abs(row_sums)),
        np.max(np.abs(col_sums)),
    )
    w = len(str(max_val)) + 1

    # print matrix with row sums
    for i in range(n):
        row = "".join(f"{A[i,j]:>{w}}" for j in range(n))
        print(row + " │" + f"{row_sums[i]:>{w}}")

    # separator
    print("─" * (w * n) + "─┼" + "─" * w)

    # column sums
    col_row = "".join(f"{col_sums[j]:>{w}}" for j in range(n))
    print(col_row)

    # diagonals
    print()
    print(f"main diagonal sum = {diag_main}")
    print(f"anti diagonal sum = {diag_anti}\n")


prime_magic_square(4, 2026)

Feasible solution found for N = 4 

   89 1613  317    7 │ 2026
  263   43 1619  101 │ 2026
 1621  347   17   41 │ 2026
   53   23   73 1877 │ 2026
─────────────────────┼─────
 2026 2026 2026 2026

main diagonal sum = 2026
anti diagonal sum = 2026



In [ ]:
# See which values we can produce magic squares in.
for N in range(2, 17, 2):
    prime_magic_square(N, 2026)

Infeasible. No solution for N = 2 

Feasible solution found for N = 4 

 1019  167  647  193 │ 2026
  293  181 1031  521 │ 2026
  709  797   17  503 │ 2026
    5  881  331  809 │ 2026
─────────────────────┼─────
 2026 2026 2026 2026

main diagonal sum = 2026
anti diagonal sum = 2026

Feasible solution found for N = 6 

  541  503  307   29  263  383 │ 2026
  761  113  193   47   53  859 │ 2026
  101   97   71  769  977   11 │ 2026
  163  617  709  233  281   23 │ 2026
  353    5  739  349  449  131 │ 2026
  107  691    7  599    3  619 │ 2026
───────────────────────────────┼─────
 2026 2026 2026 2026 2026 2026

main diagonal sum = 2026
anti diagonal sum = 2026

Feasible solution found for N = 8 

   73   89  163  929  347  101  313   11 │ 2026
  227 1153   23   29  281   59   31  223 │ 2026
   37   47   67  137  103 1481    3  151 │ 2026
  691  331   71  239    7   97  307  283 │ 2026
  337  109  263  127  139   41  997   13 │ 2026
  367  107  269  113  599   19    5  547 │ 2026
  211 

Since we have exhausted all possible sizes of prime magic squares with sum 2026, we know that only sizes of $\boxed{N = 4, 6, 8}$ are capable of producing 2026-sum magic squares.
